In [1]:
%pip install llama-index-embeddings-huggingface

Note: you may need to restart the kernel to use updated packages.


In [2]:
from llama_index.core import Settings, VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama

# 환경 변수 설정
import os
from dotenv import load_dotenv

load_dotenv()

True

In [8]:
# Ollama LLM 설정
llm = Ollama(
    model="llama3.2:3b",  # Ollama에서 제공하는 Llama2 3B 모델
    temperature=0.1,  # 응답의 창의성 조절 (0에 가까울수록 일관된 응답)
    context_window=4096,  # 컨텍스트 윈도우 크기
    timeout=120,  # 타임아웃 시간을 120초로 설정
    request_timeout=120,  # 요청 타임아웃도 120초로 설정
    additional_kwargs={  # 추가 파라미터
        "num_gpu": 0,  # CPU 사용 설정
        "seed": 42,  # 재현성을 위한 시드 설정
    },
)

# BGE 임베딩 모델 설정
embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-large-zh", device="cpu"  # CPU 사용. GPU의 경우 "cuda"
)

In [5]:
# ServiceContext 설정
Settings.llm = llm
Settings.embed_model = embed_model

# PDF 문서 로드
documents = SimpleDirectoryReader(
    input_files=[
        "/Users/sangwoo/Desktop/PYTHON BASIC/LLAMA_INDEX/LlamaIndex_Codes/data/240828_(AI리포트)_미국의_인공지능(AI)_정책,전략.pdf"
    ],
    filename_as_id=True,
).load_data()

In [6]:
# 인덱스 생성
index = VectorStoreIndex.from_documents(documents)

# 쿼리 엔진 생성
query_engine = index.as_query_engine(
    streaming=True,  # 스트리밍 응답 활성화
    similarity_top_k=3,  # 상위 3개의 가장 관련성 높은 문서 청크 사용
)

In [10]:
# 질문하기
response = query_engine.query(
    """
    책임 있는 AI 개발에서 미국의 리더십을 명확히 하는 데 
    AI안전 연구소가 맡은 중요한 역할이 있다고 생각한다고 말한 사람과 
    그 사람이 누구와 협력한다고 했는지 알려줘
    """
)
print(response)

오픈AI와 앤솔로픽(Anthropic)이 미국 상무부 산하 국립표준기술연구소(NIST)에 설치된 AI안전연구소(USAISI)와 계약을 체결하여, AI 기술의 안전하고 윤리적인 이용을 둘러싸고 규제당국의 엄격한 감시를 받는 상황으로, 그 사람이 미국의 리더십을 명확히 하는 데 중요한 역할을 하고 있다고 말한 사람은 오픈AI와 앤솔로픽(Anthropic)의 대표인 것으로 보인다.
